In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

# Hyperparameters

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Loss Function

In [6]:
loss_fn = nn.CrossEntropyLoss()

# Optimizer

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Full Implementation

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298229  [   64/60000]
loss: 2.291261  [ 6464/60000]
loss: 2.272994  [12864/60000]
loss: 2.263923  [19264/60000]
loss: 2.245935  [25664/60000]
loss: 2.211690  [32064/60000]
loss: 2.213151  [38464/60000]
loss: 2.180719  [44864/60000]
loss: 2.168194  [51264/60000]
loss: 2.143577  [57664/60000]
Test Error: 
 Accuracy: 48.2%, Avg loss: 2.142059 

Epoch 2
-------------------------------
loss: 2.143471  [   64/60000]
loss: 2.142586  [ 6464/60000]
loss: 2.087763  [12864/60000]
loss: 2.105677  [19264/60000]
loss: 2.045928  [25664/60000]
loss: 1.984819  [32064/60000]
loss: 2.000513  [38464/60000]
loss: 1.929740  [44864/60000]
loss: 1.923086  [51264/60000]
loss: 1.853481  [57664/60000]
Test Error: 
 Accuracy: 59.2%, Avg loss: 1.862349 

Epoch 3
-------------------------------
loss: 1.886529  [   64/60000]
loss: 1.866915  [ 6464/60000]
loss: 1.753535  [12864/60000]
loss: 1.792023  [19264/60000]
loss: 1.672882  [25664/60000]
loss: 1.631400  [32064/600